In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/My\ Drive/MachineTranslation

/content/gdrive/My Drive/MachineTranslation


In [0]:
import pandas as pd
import numpy as np

Seed is added to get same random order every time we run code

In [4]:
df_train = pd.read_csv("train.csv")
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


For torchtext, we need to remove '\n'

In [0]:
VAL_RATIO = 0.2
def prepare_csv(seed=999):
    df_train = pd.read_csv("train.csv")
    df_train["comment_text"] = \
        df_train.comment_text.str.replace("\n", " ")

    df_train = df_train[:30000]
    idx = np.arange(df_train.shape[0])
    np.random.seed(seed)
    np.random.shuffle(idx)
    val_size = int(len(idx) * VAL_RATIO)
    df_train.iloc[idx[val_size:], :].to_csv(
        "dataset_train.csv", index=False)
    df_train.iloc[idx[:val_size], :].to_csv(
        "dataset_val.csv", index=False)
    df_test = pd.read_csv("test.csv")
    df_test["comment_text"] = \
        df_test.comment_text.str.replace("\n", " ")

    df_test = df_test[:2000]
    df_test.to_csv("dataset_test.csv", index=False)

In [0]:
prepare_csv()

In [22]:
dataset_train = pd.read_csv('dataset_train.csv')
dataset_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1427a4ac2f0992eb,""" Your request at Files for upload Comment:...",0,0,0,0,0,0
1,15ea0c2050f01025,"""==Request for Usurpation== Hello, Mtx. A req...",0,0,0,0,0,0
2,2a23297371c49a13,What a bunch of crap. If there is controversy ...,1,0,0,0,0,0
3,173c41c69e795d55,""" Hi, . I saw your revisions here and here. Th...",0,0,0,0,0,0
4,1ca2f588aff05826,"""I think that's what led to an inaccuracy 198...",0,0,0,0,0,0


In [8]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
import re
import spacy
NLP = spacy.load('en')
MAX_CHARS = 20000
def tokenizer(comment):
    comment = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", 
        str(comment))
    comment = re.sub(r"[ ]+", " ", comment)
    comment = re.sub(r"\!+", "!", comment)
    comment = re.sub(r"\,+", ",", comment)
    comment = re.sub(r"\?+", "?", comment)
    if (len(comment) > MAX_CHARS):
        comment = comment[:MAX_CHARS]
    return [
        x.text for x in NLP.tokenizer(comment) if x.text != " "]

In [0]:
import torch
from torchtext import data


comment = data.Field(
        sequential=True,
        fix_length=None,
        tokenize=tokenizer,
        pad_first=False,
        lower=True
    )

train, val = data.TabularDataset.splits(
        path='', format='csv', skip_header=True,
        train='dataset_train.csv', validation='dataset_val.csv',
        fields=[
            ('id', None),
            ('comment_text', comment),
            ('toxic', data.Field(
                use_vocab=False, sequential=False)),
            ('severe_toxic', data.Field(
                use_vocab=False, sequential=False)),
            ('obscene', data.Field(
                use_vocab=False, sequential=False)),
            ('threat', data.Field(
                use_vocab=False, sequential=False)),
            ('insult', data.Field(
                use_vocab=False, sequential=False)),
            ('identity_hate', data.Field(
                use_vocab=False, sequential=False)),
        ])

test = data.TabularDataset(
        path='dataset_test.csv', format='csv', 
        skip_header=True,
        fields=[
            ('id', None),
            ('comment_text', comment)
        ])


comment.build_vocab(
        train, val, test,
        max_size=20000,
        min_freq=50
    )

In [40]:
print(len(train.examples))
print(len(val.examples))
print(len(test.examples))

24000
6000
2000


In [41]:
print(len(comment.vocab))

3327


In [42]:
for i in range(5):
  print(vars(train.examples[i]))

{'comment_text': ['your', 'request', 'at', 'files', 'for', 'upload', 'comment', "'", "'", "'", "'", "''", "'", "'", "'", "'"], 'toxic': '0', 'severe_toxic': '0', 'obscene': '0', 'threat': '0', 'insult': '0', 'identity_hate': '0'}
{'comment_text': ['request', 'for', 'usurpation', 'hello', ',', 'mtx', '.', 'a', 'request', 'has', 'been', 'made', 'at', 'wikipedia', 'changing', 'username', 'usurpations', 'to', 'usurp', ',', 'or', 'take', 'over', ',', 'your', 'username', 'because', 'you', 'have', 'not', 'used', 'it', 'to', 'edit', ',', 'and', 'another', 'user', 'would', 'like', 'to', 'use', 'it', 'to', 'edit', '.', 'wikipedia', 'changing', 'username', 'usurpations', 'says', 'that', 'any', 'unused', 'username', 'may', 'be', 'usurped', 'if', 'the', 'user', 'is', 'given', 'seven', 'days', 'notice', 'to', 'object', 'and', 'does', 'not', 'do', 'so', '.', 'if', 'you', 'have', 'an', 'email', 'set', 'in', 'your', 'preferences', ',', 'you', 'should', 'be', 'getting', 'an', 'email', 'from', 'a', 'bure

In [0]:
from torchtext.data import Field, BucketIterator

In [0]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = BucketIterator.splits(
    (train, val), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.comment_text),
     device = device)

In [45]:
len(train_iterator)

188

In [0]:
test_iterator = data.Iterator(
        test, batch_size=BATCH_SIZE, device=device,
        sort_within_batch = True,
        sort_key = lambda x : len(x.comment_text),
    )

In [47]:
print(len(test_iterator))

16


In [48]:
for examples in train_iterator:
  x = examples.comment_text # (fix_length, batch_size) Tensor
  y = torch.stack([
        examples.toxic, examples.severe_toxic, 
        examples.obscene,
        examples.threat, examples.insult, 
        examples.identity_hate
    ], dim=1)
  
  print(x.shape)
  print(y.shape)
  break

torch.Size([31, 128])
torch.Size([128, 6])
